In [34]:
from sqlalchemy import create_engine, ForeignKey, String

engine = create_engine("sqlite+pysqlite:///database.sqlite", echo=True)

# 建立声明性基础
from sqlalchemy.orm import DeclarativeBase


class Base(DeclarativeBase):
    pass


# 声明映射类
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship


class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [35]:
u1 = User(name="pkrabs", fullname="Pearl Krabs")

In [36]:
u1.addresses

[]

In [37]:
a1 = Address(email_address="pearl.krabs@gmail.com")

In [38]:
u1.addresses.append(a1)

In [39]:
u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com')]

In [40]:
a1.user

User(id=None, name='pkrabs', fullname='Pearl Krabs')

In [41]:
a2 = Address(email_address="pearl@aol.com", user=u1)

In [42]:
u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com'),
 Address(id=None, email_address='pearl@aol.com')]

In [43]:
a2.user

User(id=None, name='pkrabs', fullname='Pearl Krabs')

In [44]:
from sqlalchemy.orm import Session

session = Session(engine)

In [52]:
session.add(u1)

In [54]:
u1 in session

True

In [55]:
a1 in session

True

In [56]:
a2 in session

True

In [58]:
session.commit()

2024-12-25 17:30:19,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-25 17:30:19,886 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-12-25 17:30:19,886 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('pkrabs', 'Pearl Krabs')
2024-12-25 17:30:19,889 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURNING id
2024-12-25 17:30:19,890 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('pearl.krabs@gmail.com', 3)
2024-12-25 17:30:19,891 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURNING id
2024-12-25 17:30:19,892 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('pearl@aol.com', 3)
2024-12-25 17:30:19,893 INFO sqlalchemy.engine.Engine COMMIT


In [59]:
print(u1.id)

3


In [60]:
print(a1.user.id)

3


In [64]:
print(a2.user.id)

3


In [66]:
a1

Address(id=7, email_address='pearl.krabs@gmail.com')

In [69]:
a2

Address(id=8, email_address='pearl@aol.com')

In [71]:
from sqlalchemy import select

print(select(Address.email_address).select_from(User).join(User.addresses))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


### 加载器策略 (延迟加载)

In [78]:
from sqlalchemy.orm import selectinload

for user_obj in session.execute(select(User).options(selectinload(User.addresses))).scalars():
    print("#############################")
    print(user_obj.addresses)  # access addresses collection already loaded

2024-12-25 17:37:04,069 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-12-25 17:37:04,070 INFO sqlalchemy.engine.Engine [cached since 131.1s ago] ()
2024-12-25 17:37:04,072 INFO sqlalchemy.engine.Engine SELECT address.user_id AS address_user_id, address.id AS address_id, address.email_address AS address_email_address 
FROM address 
WHERE address.user_id IN (?, ?, ?)
2024-12-25 17:37:04,073 INFO sqlalchemy.engine.Engine [cached since 131.1s ago] (1, 2, 3)
#############################
[Address(id=1, email_address='spongebob@sqlalchemy.org'), Address(id=4, email_address='spongebob@sqlalchemy.org')]
#############################
[Address(id=2, email_address='sandy@sqlalchemy.org'), Address(id=3, email_address='sandy@squirrelpower.org'), Address(id=5, email_address='sandy@sqlalchemy.org'), Address(id=6, email_address='sandy@squirrelpower.org')]
#############################
[Address(id=7, email_address='pearl.krabs@gma

In [79]:
from sqlalchemy.orm import selectinload
stmt = select(User).options(selectinload(User.addresses)).order_by(User.id)
for row in session.execute(stmt):
    print(
        f"{row.User.name}  ({', '.join(a.email_address for a in row.User.addresses)})"
    )

2024-12-25 17:41:07,417 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.id
2024-12-25 17:41:07,418 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()
2024-12-25 17:41:07,420 INFO sqlalchemy.engine.Engine SELECT address.user_id AS address_user_id, address.id AS address_id, address.email_address AS address_email_address 
FROM address 
WHERE address.user_id IN (?, ?, ?)
2024-12-25 17:41:07,421 INFO sqlalchemy.engine.Engine [cached since 374.4s ago] (1, 2, 3)
squidward  (spongebob@sqlalchemy.org, spongebob@sqlalchemy.org)
ehkrabs  (sandy@sqlalchemy.org, sandy@squirrelpower.org, sandy@sqlalchemy.org, sandy@squirrelpower.org)
pkrabs  (pearl.krabs@gmail.com, pearl@aol.com)


In [80]:
from sqlalchemy.orm import joinedload
stmt = (
    select(Address)
    .options(joinedload(Address.user, innerjoin=True))
    .order_by(Address.id)
)
for row in session.execute(stmt):
    print(f"{row.Address.email_address} {row.Address.user.name}")

2024-12-25 17:46:10,856 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id, user_account_1.id AS id_1, user_account_1.name, user_account_1.fullname 
FROM address JOIN user_account AS user_account_1 ON user_account_1.id = address.user_id ORDER BY address.id
2024-12-25 17:46:10,857 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ()
spongebob@sqlalchemy.org squidward
sandy@sqlalchemy.org ehkrabs
sandy@squirrelpower.org ehkrabs
spongebob@sqlalchemy.org squidward
sandy@sqlalchemy.org ehkrabs
sandy@squirrelpower.org ehkrabs
pearl.krabs@gmail.com pkrabs
pearl@aol.com pkrabs


In [81]:
from sqlalchemy.orm import contains_eager
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "pkrabs")
    .options(contains_eager(Address.user))
    .order_by(Address.id)
)
for row in session.execute(stmt):
    print(f"{row.Address.email_address} {row.Address.user.name}")

2024-12-25 17:47:53,078 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname, address.id AS id_1, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? ORDER BY address.id
2024-12-25 17:47:53,079 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ('pkrabs',)
pearl.krabs@gmail.com pkrabs
pearl@aol.com pkrabs


In [82]:
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "pkrabs")
    .options(joinedload(Address.user))
    .order_by(Address.id)
)
print(stmt)  # SELECT has a JOIN and LEFT OUTER JOIN unnecessarily

SELECT address.id, address.email_address, address.user_id, user_account_1.id AS id_1, user_account_1.name, user_account_1.fullname 
FROM address JOIN user_account ON user_account.id = address.user_id LEFT OUTER JOIN user_account AS user_account_1 ON user_account_1.id = address.user_id 
WHERE user_account.name = :name_1 ORDER BY address.id


加载介绍网址: https://docs.sqlalchemy.org/en/20/tutorial/orm_related_objects.html